In [1]:
import matplotlib.pyplot as plt
import numpy as np
from random import randrange
from skimage.feature import local_binary_pattern

In [2]:
DATA_DIR = 'data/'
DATA_FILENAME = 'data_naive1d_subset20k'
data_train = np.load(DATA_DIR + DATA_FILENAME + '_train' + '.npz')
X_train = data_train['X']
data_train.close()
X_train.shape, X_train.dtype

((20000, 9216), dtype('uint8'))

In [3]:
# Extract (<i>, <j>)th cell as 1-D vector
def extract_cell(v, i, j):
    img = v.reshape(96, 96)
    return img[32*i:32*(i+1), 32*j:32*(j+1)].ravel()

# Turn a <v> of size 9216 (96x96) into 9 vectors of 1024 (32x32)
# by dividing image into 9 cells and un-rolling each cell
# individually
def vec_to_cells(v):
    cells = []
    for i in range(3):
        for j in range(3):
            cells.append(extract_cell(v, i, j))
    return np.array(cells)

In [4]:
X_train9 = np.vstack([vec_to_cells(sample) for sample in X_train])
X_train9.shape, X_train9.dtype

((180000, 1024), dtype('uint8'))

In [5]:
# Return smaller of either Kullback–Leibler Divergences
def kld(p, q):
    p_q = np.sum(p * np.log2(p/q))
    q_p = np.sum(q * np.log2(q/p))
#     if p_q < 0 or q_p < 0:
#         print(p); print(q); print(sum)
#         raise Exception
    return min(p_q, q_p)

In [6]:
# Local Binary Pattern Histogram
def lbp_histogram(v, dim=32):
    img = v.reshape(dim, dim)
    patterns = local_binary_pattern(img, 8, 1, 'uniform')
    hist, _ = np.histogram(patterns, bins=np.arange(59 + 1))
    # if only the first ten bins are non-zero, we can reduce our
    # landmark feature size
    assert hist[10:].sum() == 0
    # add one to avoid divide-by-zero errors later in kld()
    return hist[:10].astype('int16') + 1

In [7]:
# Reduce <X> to <target_size> by eliminating similar features,
# using <distance> as a metric, via an iterative, stochastic
# sampling of <num_subsamples> and eliminating <num_eliminate>
# features at each iteration
def reduce(X, target_size, num_subsample, num_eliminate, distance):
    modulo = 200
    while (len_X := len(X)) > target_size:
        x1, x2 = (randrange(len_X), randrange(len_X))
        distances = [(0, distance(X[x1], X[x2]))]
        for _ in range(num_subsample):
            x1, x2 = (randrange(len_X), randrange(len_X))
            if x1 == x2: continue
            dist = distance(X[x1], X[x2])
            if dist < distances[-1][1]:
                distances.append((x1, dist))
        eliminate = [tup[0] for tup in distances[-num_eliminate:]]
        X = np.delete(X, eliminate, axis=0)
        modulo -= 1
        if (modulo == 0):
            print(len_X, end=' ')
            modulo = 200
    print()
    return X

In [8]:
train_hists = np.apply_along_axis(lbp_histogram, 1, X_train9)
train_hists.shape, train_hists.dtype

((180000, 10), dtype('int16'))

In [9]:
%time landmarks = reduce(train_hists, 512, 2000, 5, kld)
%time landmarks = reduce(landmarks, 256, 2000, 1, kld)
landmarks.shape, landmarks.dtype

179016 178028 177039 176058 175069 174094 173114 172130 171146 170175 169196 168210 167225 166233 165250 164262 163280 162302 161316 160344 159366 158380 157389 156410 155435 154444 153467 152487 151509 150529 149549 148571 147582 146597 145614 144640 143658 142690 141713 140739 139758 138784 137798 136820 135839 134856 133876 132886 131896 130909 129931 128954 127967 126984 126001 125026 124046 123065 122077 121091 120102 119118 118132 117149 116166 115185 114205 113216 112226 111243 110266 109284 108296 107317 106345 105360 104376 103382 102399 101424 100449 99465 98480 97491 96508 95528 94545 93566 92584 91607 90627 89647 88671 87704 86726 85741 84761 83770 82784 81801 80818 79831 78850 77869 76890 75902 74922 73935 72948 71960 70969 69981 68995 68014 67030 66054 65079 64097 63110 62135 61152 60184 59206 58231 57247 56262 55291 54309 53334 52341 51358 50378 49398 48427 47442 46464 45470 44499 43511 42529 41555 40572 39587 38596 37613 36628 35644 34657 33679 32702 31725 30744 29772 2

((256, 10), dtype('int16'))

In [10]:
landmarks.min(), landmarks.max(), landmarks.mean()

(1, 860, 103.4)

In [11]:
landmarks

array([[  7,  40,   6, ...,  52, 481,  83],
       [ 25,  66,  19, ...,  99, 371,  81],
       [ 15,  20,  12, ...,  79, 313,  39],
       ...,
       [  3,  12,  28, ...,  28,  20,  12],
       [ 22,  40,  23, ...,  60, 524,  45],
       [ 18,  78,  10, ..., 105, 311,  90]], dtype=int16)

In [12]:
LAND_FILENAME = 'data_landmarks_subset20k'
np.savez(DATA_DIR + LAND_FILENAME, landmarks=landmarks)